In [1]:
#Basic Server

import socketserver
import threading

class TCPHandler(socketserver.BaseRequestHandler):
    def setup(self):  # 연결 설정
        print("Connected")
        
    def handle(self):  # 데이터 수신
        while True:  # 연결을 유지하려면 무한 루프 실행
            try:
                self.data = self.request.recv(1024).strip()
                if not self.data:
                    return
            except:
                self.finish()
                return
            print("{} wrote:".format(self.client_address[0]))
            print(self.data)

            # 수신 메시지 전송
            self.request.sendall(self.data.upper())
    
    def finish(self):  # 연결 해제
        print("Disconnected")
        self.request.close()

if __name__ == "__main__":
    HOST, PORT = "", 2500
    server = socketserver.TCPServer((HOST, PORT), TCPHandler, bind_and_activate=True)

    # 서버를 별도의 스레드에서 실행
    server_thread = threading.Thread(target=server.serve_forever)
    server_thread.daemon = True  # 메인 스레드 종료 시 함께 종료
    server_thread.start()

    print("Server running in thread", server_thread.name)


Server running in thread Thread-5 (serve_forever)


Connected
127.0.0.1 wrote:
b'asd'
127.0.0.1 wrote:
b'ds'
127.0.0.1 wrote:
b'sd'
127.0.0.1 wrote:
b'sdds'
Disconnected
Disconnected
Connected
127.0.0.1 wrote:
b'as'
127.0.0.1 wrote:
b'asd'
127.0.0.1 wrote:
b'asdasd'
Disconnected
Disconnected
Connected
127.0.0.1 wrote:
b'asdd'
127.0.0.1 wrote:
b'asdasd'
127.0.0.1 wrote:
b'dksdfsd'
127.0.0.1 wrote:
b'sd'
127.0.0.1 wrote:
b'fffffffffffffffffffffffffffffffff'
127.0.0.1 wrote:
b'fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'
127.0.0.1 wrote:
b'ffffffffffffffffffffffffffffffffffffffffffffffffffffffff'
127.0.0.1 wrote:
b'sdf'
127.0.0.1 wrote:
b's'
127.0.0.1 wrote:
b'df'
127.0.0.1 wrote:
b'sdf'
127.0.0.1 wrote:
b'sdf'
127.0.0.1 wrote:
b's'
127.0.0.1 wrote:
b'df'
127.0.0.1 wrote:
b'sdf'
127.0.0.1 wrote:
b'fsd'
127.0.0.1 wrote:
b'sfd'
127.0.0.1 wrote:
b'sfd'
127.0.0.1 wrote:
b'sfd'
127.0.0.1 wrote:
b'sfd'
127.0.0.1 wrote:
b'sdf'
127.0.0.1 wrote:
b'sdf'
Disconnected
Disconnected
Connected
127.0.0.1 wrote:
b'asdasdasd'
127.0.0

In [1]:
import socketserver
import threading
import sqlite3

# SQLite 데이터베이스 초기화
def initialize_database():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        username TEXT PRIMARY KEY,
                        password TEXT NOT NULL)''')
    # 예제 사용자 추가
    cursor.execute('INSERT OR IGNORE INTO users (username, password) VALUES (?, ?)', ('user1', 'password1'))
    cursor.execute('INSERT OR IGNORE INTO users (username, password) VALUES (?, ?)', ('user2', 'password2'))
    conn.commit()
    conn.close()

initialize_database()

class TCPHandler(socketserver.BaseRequestHandler):
    def setup(self):
        print("Connected")

    def handle(self):
        while True:
            try:
                self.data = self.request.recv(1024).strip()
                if not self.data:
                    return
            except:
                self.finish()
                return

            credentials = self.data.decode().split(":")
            if len(credentials) == 2:
                username, password = credentials
                if self.authenticate_user(username, password):
                    self.request.sendall(b"True")
                else:
                    self.request.sendall(b"False")
            else:
                self.request.sendall(b"False")

    def authenticate_user(self, username, password):
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
        user = cursor.fetchone()
        conn.close()
        return user is not None

    def finish(self):
        print("Disconnected")
        self.request.close()

if __name__ == "__main__":
    HOST, PORT = "", 2500
    server = socketserver.TCPServer((HOST, PORT), TCPHandler, bind_and_activate=True)

    server_thread = threading.Thread(target=server.serve_forever)
    server_thread.daemon = True
    server_thread.start()

    print("Server running in thread", server_thread.name)


Server running in thread Thread-5 (serve_forever)


Connected
Disconnected
Connected
Disconnected


In [ ]:
import socketserver
import threading

clients = {}

class TCPHandler(socketserver.BaseRequestHandler):
    def setup(self):
        print(f"Connected: {self.client_address[0]}")
        self.request.send("Welcome! Please enter your nickname: ".encode())
        self.nickname = self.request.recv(1024).strip().decode()
        clients[self.request] = self.nickname
        self.broadcast(f"{self.nickname} has joined the chat!")

    def handle(self):
        while True:
            try:
                data = self.request.recv(1024).strip()
                if not data:
                    break
                message = f"{self.nickname}: {data.decode()}"
                print(message)
                self.broadcast(message, exclude=self.request)
            except:
                break

    def finish(self):
        print(f"Disconnected: {self.nickname}")
        if self.request in clients:
            del clients[self.request]
        self.broadcast(f"{self.nickname} has left the chat.")
        self.request.close()

    def broadcast(self, message, exclude=None):
        for client in clients:
            if client != exclude:
                try:
                    client.send(message.encode())
                except:
                    pass

if __name__ == "__main__":
    HOST, PORT = "", 2500
    server = socketserver.ThreadingTCPServer((HOST, PORT), TCPHandler)

    print(f"Server running on port {PORT}")
    server.serve_forever()


Server running on port 2500
Connected: 127.0.0.1
1: asd
Connected: 127.0.0.1
Disconnected: 1
asd: d
asd: d
asd: a
asd: s
asd: d
asd: a
asd: sd
Connected: 127.0.0.1
h: asd
h: dd
h: assssssssssssss
Connected: 127.0.0.1
name: dsd
Disconnected: h
Disconnected: name
Connected: 127.0.0.1
sss: ds
sss: sdasdasd
Connected: 127.0.0.1
aaaaaaaaaaa: dsd
aaaaaaaaaaa: asd
sss: asd
Connected: 127.0.0.1
Disconnected: sss
asdddddd: djs안녕
asdddddd: 안녕하시오
asdddddd: 인알
asdddddd: ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
asdddddd: ㅇ
Connected: 127.0.0.1
Disconnected: asdddddd
Connected: 127.0.0.1
Disconnected: ㅇㅁㄴㅇ
ㅁㄴㅇㄴㅁ: ㅇㅇㅇㅇㅇ
ㅁㄴㅇㄴㅁ: ㅁㄴㅇ
ㅁㄴㅇㄴㅁ: ㄴㄴㄴ
ㅁㄴㅇㄴㅁ: sdasd
ㅁㄴㅇㄴㅁ: ads
ㅁㄴㅇㄴㅁ: d
ㅁㄴㅇㄴㅁ: ds
ㅁㄴㅇㄴㅁ: sd
ㅁㄴㅇㄴㅁ: d
Connected: 127.0.0.1
asd: sd
Disconnected: ㅁㄴㅇㄴㅁ
asd: asd
Connected: 

Server running in thread Thread-5 (serve_forever)
Connected
